In [1]:
# Fusion ART Model for Professor Data using Word2Vec
# Python fusionART.py code by Dr. Budhitama Subagdja, modified for this dataset by Patrick Tjahjadi

# ----- Imported Libraries -----
from fusionART import *

from collections import defaultdict
from collections import Counter

import pandas as pd

import math

import random

from numpy import array
from numpy import argmax

from itertools import islice

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk import ngrams
from nltk.corpus import stopwords

import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import gensim.downloader as api
from gensim import corpora

import tensorflow as tf
from tensorflow.keras import layers, models

import re

import csv

import wikipedia

import dill

import transformers
# -------------------------------

data = pd.read_csv('SCSE ProfProfile.csv', encoding='ISO-8859-1')

In [2]:
# Function to convert 1D lists into 2D, based on the number of elements per list.
def convert_to_2d(list_to_convert, list_per_entry): 
    it = iter(list_to_convert) 
    return [list(islice(it, i)) for i in list_per_entry] 

In [3]:
# Function to clean words from punctuation and remove capital case to standardise text tokens
def clean_text(word):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_123456789~'''
    no_punct = ""
    for char in word:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct.lower()

In [4]:
# Function accepts a vector and normalise its values to a range between 0 and 1
def normalise_vector(vector):
    normalised_vector = [0] * len(vector)
    max_value = max(vector)
    min_value = min(vector)
    for index in range(0, len(vector)):
        normalised_vector[index] = (vector[index] - min_value) / (max_value - min_value)
    return normalised_vector

In [5]:
# Function to search for n-grams for n up to 6 and return the n-grammed text.
# Functions accepts list of n-grams to be formed, the text and the common n-grams within the text, respectively.
def search_for_ngrams(ngram_list, text_list):
    ngrammed_text = []
    for i in range(0, len(text_list)):
        bigram = text_list[i-1].lower()+"_"+text_list[i].lower()
        trigram = text_list[i-2].lower()+"_"+text_list[i-1]+"_"+text_list[i].lower()
        fourgram = text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        fivegram = text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        sixgram = text_list[i-5].lower()+"_"+text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        if bigram in ngram_list:
            ngrammed_text.append(bigram)
        if trigram in ngram_list:
            ngrammed_text.append(trigram)
        if fourgram in ngram_list:
            ngrammed_text.append(fourgram)
        if fivegram in ngram_list:
            ngrammed_text.append(fourgram)
        if sixgram in ngram_list:
            ngrammed_text.append(fourgram)
        ngrammed_text.append(text_list[i])
    return ngrammed_text


In [6]:
# Convert each attribute of the dataset to separate lists
name = []
group = []
university = []
research_interest = []
for idx, row in data.iterrows():
    name.append(row[0])
    group.append(row[1])
    university.append(row[2])
    research_interest.append(row[3])

# Problem: research_interest and university uses semicolons to denote multiple interests and universities
# Change their format to a list of universities and research interests of a person

university_preprocessed = []
for entry in university:
    uni = entry.split(";")
    university_preprocessed.append(uni)

research_interest_preprocessed = []
for entry in research_interest:
    interests = entry.split(";")
    research_interest_preprocessed.append(interests)    

    
# Break university_processed and research_interest_processed to 1D lists for one hot encoding
university_1d = []
for universities in university_preprocessed:
    for uni in universities:
        university_1d.append(uni)
        
research_interest_1d = []
for interests in research_interest_preprocessed:
    for interest in interests:
        research_interest_1d.append(interest)
        

In [7]:
# Determine the number of universities and research interests per professor

universities_per_person = []
for universities in university_preprocessed:
    universities_per_person.append(len(universities))
    

research_interest_per_person = []
for research_interests in research_interest_preprocessed:
    research_interest_per_person.append(len(research_interests))
    

In [8]:
# Research interest preprocessing, to consider each research keywords as a phrase
for index in range(0, len(research_interest_1d)):
    research_interest_1d[index] = research_interest_1d[index].replace(" ", "_")
    research_interest_1d[index] = research_interest_1d[index].lower()
    research_interest_1d[index] = research_interest_1d[index].replace(":", "")
    research_interest_1d[index] = research_interest_1d[index].replace(",", "")


In [9]:
# Create a list of unique names, groups, universities and research interests to define the model schema
# (I refrained from using sets because it would randomise the one-hot ordering)
unique_name = []
unique_group = []
unique_university = []
unique_research_interest = []
[unique_name.append(i) for i in name if i not in unique_name]
[unique_group.append(i) for i in group if i not in unique_group]
[unique_university.append(i) for i in university_1d if i not in unique_university]
[unique_research_interest.append(i) for i in research_interest_1d if i not in unique_research_interest]
pass


In [10]:
# One Hot Encoding for name, group and university
name_label_encoder = LabelEncoder()
uni_label_encoder = LabelEncoder()
group_label_encoder = LabelEncoder()
research_label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder(sparse=False)

name_integer_encoded = name_label_encoder.fit_transform(name)
name_integer_encoded = name_integer_encoded.reshape(len(name_integer_encoded), 1)
name_onehot = onehot_encoder.fit_transform(name_integer_encoded)

group_integer_encoded = group_label_encoder.fit_transform(group)
group_integer_encoded = group_integer_encoded.reshape(len(group_integer_encoded), 1)
group_onehot = onehot_encoder.fit_transform(group_integer_encoded)

university_integer_encoded = uni_label_encoder.fit_transform(array(university_1d))
university_integer_encoded = university_integer_encoded.reshape(len(university_integer_encoded), 1)
university_onehot = onehot_encoder.fit_transform(university_integer_encoded)


In [11]:
# Convert the numpy arrays to lists
name_onehotlist = name_onehot.tolist()
group_onehotlist = group_onehot.tolist()
university_onehotlist = university_onehot.tolist()



In [12]:
# Problem: each professor has multiple universities
# Convert these 1D arrays to 2D, to associate them to each professor

university_onehotlist_processed = []

university_onehotlist_processed = convert_to_2d(university_onehotlist, universities_per_person)

In [13]:
# Create an array that allows multiple universities per professor by summation
university_2d = []
for universities_per_professor in university_onehotlist_processed:
    university_2d.append([sum(x) for x in zip(*universities_per_professor)])
    

# Since universities can be duplicate, change so that the maximum value of each university is 1
for universities_per_professor in university_2d:
    for index in range(0, len(universities_per_professor)):
        if universities_per_professor[index] > 1:
            universities_per_professor[index] = 1.0


In [14]:
# Gather text tokens from Wikipedia articles for all research keywords 
wiki_text = []
for keyword in research_interest_1d:
    try:
        p = wikipedia.page(keyword)
        wiki_text.append(p.content.split())
        
    # Include exception handling if the Wikipedia article can not be found
    except wikipedia.exceptions.PageError:
        pass
    except wikipedia.DisambiguationError:
        pass



C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [15]:
# Remove common stop words from the retrieved Wikipedia articles
eng_stopwords = set(stopwords.words('english'))
for text in wiki_text:
    for word in text:
        if word in eng_stopwords:
            text.remove(word)

In [16]:
# Search for research keyword n-grams from Wikipedia articles and find most common n-grams

trigger = 0
for text in wiki_text:
    string = " ".join(text)
    if (trigger == 0):
        bigram_counts = Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts = Counter(ngrams(string.split(), 3)).most_common(10)
        trigger += 1
    else:
        bigram_counts += Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts += Counter(ngrams(string.split(), 3)).most_common(10)




In [17]:
# Create a list of common bigrams and trigrams from the Wikipedia articles
common_ngrams = bigram_counts + trigram_counts
common_ngram_list = []
for each_ngram in common_ngrams:
    common_ngram_list.append("_".join(each_ngram[0]))


In [18]:
# Search for research keyword n-grams from the Wikipedia articles and clean them
ngram_wiki_tokens = []

for text in wiki_text:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_wiki_tokens.append(search_for_ngrams(unique_research_interest + common_ngram_list, cleaned_text))

In [19]:
# Open the custom corpus text file and tokenise them. Group them with 2500 words each
testtext = []
with open('it_text.txt','r', encoding="utf-8") as f:
    for line in f:
        for word in line.split():
            testtext.append(word)  
            

# Remove stopwords in the custom text corpus
for word in testtext:
    if word in eng_stopwords:
        testtext.remove(word)
        
test_tokens = convert_to_2d(testtext, [2500] * (math.ceil(len(testtext)/2500)))


In [20]:
# Search for research keyword n-grams from the custom corpus and find most common n-grams

trigger = 0
for text in test_tokens:
    string = " ".join(text)
    if (trigger == 0):
        bigram_counts = Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts = Counter(ngrams(string.split(), 3)).most_common(10)
        trigger += 1
    else:
        bigram_counts += Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts += Counter(ngrams(string.split(), 3)).most_common(10)


In [21]:
# Create a list of common bigrams and trigrams from the custom corpus
common_ngrams = bigram_counts + trigram_counts
common_ngram_list = []
for each_ngram in common_ngrams:
    common_ngram_list.append("_".join(each_ngram[0]))

In [22]:
# Search for research keyword n-grams from the custom corpus and clean them
ngram_test_tokens = []

for text in test_tokens:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_test_tokens.append(search_for_ngrams(unique_research_interest + common_ngram_list, cleaned_text))

In [23]:
# Build the BERT model from the Transformers library

tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

nlp = transformers.TFBertModel.from_pretrained('distilbert-base-uncased')



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFBertModel: ['activation_13', 'distilbert', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Create the BERT corpus using Wikipedia articles and the custom text corpus
txt = " "
corpus = (ngram_wiki_tokens) + (ngram_test_tokens)
maxlen = 50

# add special tokens
maxqnans = np.int((maxlen-20)/2)
corpus_tokenized = ["[CLS] "+
             " ".join(tokenizer.tokenize(re.sub(r'[^\w\s]+|\n', '', 
             str(txt).lower().strip()))[:maxqnans])+
             " [SEP] " for text in corpus]

# generate masks
masks = [[1]*len(txt.split(" ")) + [0]*(maxlen - len(
           txt.split(" "))) for txt in corpus_tokenized]
    
# npadding
txt2seq = [txt + " [PAD]"*(maxlen-len(txt.split(" "))) if len(txt.split(" ")) != maxlen else txt for txt in corpus_tokenized]
    
# generate idx
idx = [tokenizer.encode(seq.split(" ")) for seq in txt2seq]
    
# generate segments
segments = [] 
for seq in txt2seq:
    temp, i = [], 0
    for token in seq.split(" "):
        temp.append(i)
        if token == "[SEP]":
             i += 1
    segments.append(temp)
# feature matrix
X_train = [np.asarray(idx, dtype='int32'), 
           np.asarray(masks, dtype='int32'), 
           np.asarray(segments, dtype='int32')]

In [25]:
# BERT feature matrix
i = 0
print("tokenized:", [tokenizer.convert_ids_to_tokens(idx) for idx in X_train[0][i].tolist()])
print("idx: ", X_train[0][i])
print("mask: ", X_train[1][i])
print("segment: ", X_train[2][i])

tokenized: ['[CLS]', '[CLS]', '[UNK]', '[SEP]', '[UNK]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[SEP]']
idx:  [101 101 100 102 100   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 102]
mask:  [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
segment:  [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [26]:
# Establish the BERT model
# inputs
idx = layers.Input((50), dtype="int32", name="input_idx")
masks = layers.Input((50), dtype="int32", name="input_masks")

# pre-trained bert with config
config = transformers.DistilBertConfig(dropout=0.2, 
           attention_dropout=0.2)
config.output_hidden_states = False
nlp = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)
bert_out = nlp(idx, attention_mask=masks)[0]
## fine-tuning
x = layers.GlobalAveragePooling1D()(bert_out)
x = layers.Dense(64, activation="relu")(x)
y_out = layers.Dense(len(np.unique(corpus)), 
                     activation='softmax')(x)

# compile
bertmodel = models.Model([idx, masks], y_out)
for layer in bertmodel.layers[:3]:
    layer.trainable = False
bertmodel.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
bertmodel.summary()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_idx (InputLayer)          [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB ((None, 50, 768),)   66362880    input_idx[0][0]                  
__________________________________________________________________________________________________
global_average_pooling1d (Globa (None, 768)          0           tf_distil_bert_model[0][0]       
______________________________________________________________________________________________

In [27]:
# Preprocess the research interest array, per professor
for prof_interests in research_interest_preprocessed:
    for interest in range(0, len(prof_interests)):
        prof_interests[interest] = prof_interests[interest].replace(" ", "_")
        prof_interests[interest] = prof_interests[interest].lower()
        prof_interests[interest] = prof_interests[interest].replace(":", "")
        prof_interests[interest] = prof_interests[interest].replace(",", "")


In [28]:
research_interest_2d = []

for prof_interests in research_interest_preprocessed:
    interest_string = " ".join(prof_interests)
    input_ids = np.array(tokenizer.encode(interest_string))[None]  
    embedding = nlp(input_ids)
    research_interest_2d.append(embedding[0][0][0][0:len(unique_research_interest)].numpy().tolist())

In [29]:
# Normalise the values of all research keyword vectors to a range between 0 and 1
normalised_research_interest_2d = []
for research_vector in research_interest_2d:
    normalised_research_interest_2d.append(normalise_vector(research_vector))

In [30]:
dill.dump_session('notebook_env_BERT.db')

TypeError: can't pickle tensorflow.python._tf_stack.StackSummary objects

In [31]:
# Define the schema for the FusionART model
model_schema = [{'name': data.columns[0], 'compl': False, 'attrib': sorted(unique_name)}, 
                {'name': data.columns[1], 'compl': False, 'attrib': sorted(unique_group)},
                {'name': data.columns[2], 'compl': False, 'attrib': sorted(unique_university)},
                {'name': data.columns[3], 'compl': True, 'attrib': sorted(unique_research_interest)}]

In [32]:
# Initialise the FusionART model
model = FusionART(schema = model_schema, beta = [1.0, 1.0, 1.0, 1.0], alpha = [0.1, 0.1, 0.1, 0.1], 
               gamma = [0.25, 0.25, 0.25, 0.25], rho = [1, 1, 1, 1])
model.F1Fields

[{'name': 'name',
  'compl': False,
  'attrib': ['AS Madhukumar',
   'Alexei Sourin',
   'Anupam Chattopadhyay',
   'Anwitaman Datta',
   'Arijit Khan',
   'Arvind Easwaran',
   'Bo An',
   'Cai Jianfei',
   'Cai Wentong',
   'Cham Tat Jen',
   'Chan Syin',
   'Chia Liang Tien Clement',
   'Chng Eng Siong',
   'Deepu Rajan',
   'Deng Ruilong',
   'Douglas Leslie Maskell',
   'Dusit Niyato',
   'Eric Cambria',
   'Gao Cong',
   'Goh Wooi Boon',
   'Guan Cuntai',
   'He Ying',
   'Huang Shell Ying',
   'Hui Siu Cheung',
   'Jagath C. Rajapakse',
   'Ke Yiping Kelly',
   'Kong Wai Kin Adams',
   'Kwoh Chee Keong',
   'Lam Kwok Yan',
   'Lam Siew Kei',
   'Lau Chiew Tong',
   'Lee Bu Sung Francis',
   'Li Fang Flora',
   'Li Mo',
   'Li Yi',
   'Liang Qianhui Althea',
   'Lin Feng',
   'Lin Guosheng',
   'Lin Shang Wei',
   'Lin Weisi',
   'Liu Weichen',
   'Liu Yang',
   'Loke Yuan Ren',
   'Long Cheng',
   'Loy Chen Change (Cavan)',
   'Lu Shijian',
   'Luo Jun',
   'Mahardhika Pratama',

In [33]:
# Store the data to the Fusion ART model

for i in range(0, len(name)):
    model.updateF1bySchema([{'name': data.columns[0], 'val': name_onehotlist[i]}, 
                            {'name': data.columns[1], 'val': group_onehotlist[i]}, 
                            {'name': data.columns[2], 'val': university_2d[i]}, 
                            {'name': data.columns[3], 'val': normalised_research_interest_2d[i]}])

    
    print("resonance search: ")
    J = model.resSearch()
    print("selected ", J)
    if model.uncommitted(J):
        print ('uncommitted')

    model.autoLearn(J)
    model.doReadoutAllFields(J)
                    
    
model.displayNetwork()

resonance search: 
selected  0
uncommitted
resonance search: 
selected  1
uncommitted
resonance search: 
selected  2
uncommitted
resonance search: 
selected  3
uncommitted
resonance search: 
selected  4
uncommitted
resonance search: 
selected  5
uncommitted
resonance search: 
selected  6
uncommitted
resonance search: 
selected  7
uncommitted
resonance search: 
selected  8
uncommitted
resonance search: 
selected  9
uncommitted
resonance search: 
selected  10
uncommitted
resonance search: 
selected  11
uncommitted
resonance search: 
selected  12
uncommitted
resonance search: 
selected  13
uncommitted
resonance search: 
selected  14
uncommitted
resonance search: 
selected  15
uncommitted
resonance search: 
selected  16
uncommitted
resonance search: 
selected  17
uncommitted
resonance search: 
selected  18
uncommitted
resonance search: 
selected  19
uncommitted
resonance search: 
selected  20
uncommitted
resonance search: 
selected  21
uncommitted
resonance search: 
selected  22
uncommitte

Code: 6 {'F2': 0, 'weights': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.378223

In [34]:
# Perform the research keywords query here

def query_by_research_with_noise(query_research, output_file, noise_limit):
    
    query_research = query_research.split(";")
    output_file.write("Research Keywords: ")
    output_file.write(', '.join(query_research)+"\n")

    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")

    # Convert all the research keywords into input vectors
    interest_string = " ".join(query_research)
    input_ids = np.array(tokenizer.encode(interest_string))[None]  
    embedding = nlp(input_ids)
    research_vector = embedding[0][0][0][0:len(unique_research_interest)].numpy().tolist()
    
    normalised_research_vector = normalise_vector(research_vector)
    

    # With a defined probability, flip the binary research keyword vectors due to noise
    for index in range(0, len(research_vector)):
        noise_value = random.random()
        if (noise_value < noise_limit):
            normalised_research_vector[index] = 1 - normalised_research_vector[index]
            
    # Define the complement-coded research vector
    compl_vector = []
    for bit in normalised_research_vector:
        compl_vector.append(1 - bit)
    

    model.setParam('gamma', [0,0,0,1])
    model.setParam('rho', [0,0,0,0])
    model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector, 'vcompl': compl_vector}], refresh = False)
    # model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector}])
    model.compChoice()

    # Find the nodes that have the highest F2 value
    maxF2value = model.codes[0]['F2']
    maxF2indexes = []

    for i in range(0, len(model.codes)-1):
        F2value = model.codes[i]['F2']
        if (F2value > maxF2value):
            maxF2value = F2value
            maxF2indexes = [i]
        elif (F2value == maxF2value):
            maxF2indexes.append(i)

    # Retrieve the data of the node(s) that have the highest F2 value
    output_file.write("Professor that follows these research keywords:")    
    for node in maxF2indexes:
        name_vector = model.codes[node]['weights'][0]
        for index in range(0, len(name_vector)):
            # Iterate through the vector and find the indexes that has a value of 1
            if (name_vector[index] == 1):
                # Retrieve the data from the model's 'attrib' attribute in F1 
                retrieved_name = model.F1Fields[0]['attrib'][index]
                output_file.write(retrieved_name)
    
    output_file.write("\n\n")
    return(retrieved_name)


In [35]:
# Research keywords query for 0% noise
output_file = open("output_zero_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()


In [46]:
# Research keywords query for 10% noise
output_file = open("output_ten_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.1)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [47]:
# Research keywords query for 20% noise
output_file = open("output_twenty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.2)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [48]:
# Research keywords query for 30% noise
output_file = open("output_thirty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.3)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [49]:
# Research keywords query for 40% noise
output_file = open("output_forty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.4)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [50]:
# Research keywords query for 50% noise
output_file = open("output_fifty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.5)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [51]:
# Research keywords query for 60% noise
output_file = open("output_sixty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.6)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [52]:
# Research keywords query for 70% noise
output_file = open("output_seventy_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.7)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [53]:
# Research keywords query for 80% noise
output_file = open("output_eighty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.8)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [54]:
# Research keywords query for 90% noise
output_file = open("output_ninety_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.9)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [55]:
# Research keywords query for 100% noise
output_file = open("output_hundred_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 1)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()